# 자연어 처리 NLP (Natural Language Processing)

자연어 처리는 인간의 언어 현상을 컴퓨터와 같은 기계를 이용하여 모사할 수 있도록 연구하고 이를 구현하는 인공지능의 주요 분야 중 하나다.<br>
컴퓨터 공학과 언어학과 같은 다른 학문과의 융합적인 요소도 필요하며, 컴퓨터와 인간 사이의 연결고리를 만들어주는 것이 자연어 처리 분야라고 할 수 있다.

## 자연어 처리의 주 응용분야

* Sentiment Analysis - 감성분석, 인터넷 이용자의 글에서 감정을 분류하는 작업
* Quention & Answering - QA, 지문을 읽고 주어진 질문에 대해 대답하는 분야
* 애플의 Siri와 같이 사용자의 의도를 파악하고 대화하거나 도움을 주는 작업
* Summarization 요약, Machine Translation 기계번역과 같은 작업
* Language Modeling - 언어모델링, 첫 몇 마디만 입력하면 뒤에 나올 단어 예측해서 선택만 하면 되는 스마트보드

1950년부터 자연어를 처리하려는 시도가 있었고 그 뒤로 계속 문제점을 가지고 있다가 <br>
딥러닝이 발전하면서 2014년에 seq2seq(Sequence-to-Sequence)구조가 제시되면서 기계 번역은 신경망을 기반으로 하는 것이 대세가 됨

딥러닝이 처음에 이미지 처리 분야에서 강한 면모를 보였다면 자연어 처리는 최근에 가장 진척이 많이 나간 연구 분야이기도 함